In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="qwen/qwen3-32b")

In [2]:
from typing import Annotated,List
import operator
from typing_extensions import Literal,TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage,SystemMessage


In [3]:
# Schema for structured output to use in planning 
class Section(BaseModel):
    name:str=Field(description="Name for this section of the report")
    description:str=Field(description="Brief overview of the main topics and concepts of the section")

class Sections(BaseModel):
    sections:List[Section]=Field(description="Sections of the  report")

# Augment the LLM with schema for sructured output
planner = llm.with_structured_output(Sections)

In [4]:
from langgraph.constants import Send

# Graph state
class State(TypedDict):
    topic:str 
    sections: list[Section]
    completed_sections: Annotated[list ,operator.add] # All worker write to this key in parallel
    final_report:str

class WorkerState(TypedDict):
    section:Section
    completed_sections: Annotated[list,operator.add]

C:\Users\Himanshu\AppData\Local\Temp\ipykernel_17028\1296749769.py:1: LangGraphDeprecatedSinceV10: Importing Send from langgraph.constants is deprecated. Please use 'from langgraph.types import Send' instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  from langgraph.constants import Send


In [9]:
# Nodes 
def orchestrator(state:State):
    """orchestrator that generates a plan for the report"""
    
    # Generate queries
    report_sections = planner.invoke(
        [
            SystemMessage(content = "Generate a plan for the report"),
            HumanMessage(content=f"Here is the report topic: {state['topic']}"),
        ]
    )

    print("Report Sections :",report_sections)
    return {"sections": report_sections.sections}

def llm_call(state: WorkerState):
    """Worker writes a section of the report"""

    # Generate section
    section = llm.invoke(
        [
            SystemMessage(content="write a report section following the provided name and description. Include no premable for each section"),

            HumanMessage(content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}"),

        ]
    )

    # Write the updated section to completed sections
    return {"completed_sections": [section.content]}

# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state:State):
    """Assign a worker to each section in the plan """

    # kick off section writing in parallel via Send() API
    return [Send("llm_call",{"section":s}) for s in state["section"]]

def synthesizer(state:State):
    """Synthesize full report from sections"""
    # List of completed sections
    completed_sections = state["completed_sections"]

    # Format completed section to str to use as context for final sections
    completed_report_sections= "\n\n---\n\n".join(completed_sections)

    return {"final_report":completed_report_sections}

In [13]:
from langgraph.graph import StateGraph, START,END
orchestrator_worker_builder = StateGraph(State)
from IPython.display import Image, display

## ADD the nodes
orchestrator_worker_builder.add_node("orchestrator",orchestrator)
orchestrator_worker_builder.add_node("llm_call",llm_call)
orchestrator_worker_builder.add_node("synthesizer",synthesizer)

# add edges to connect nodes
orchestrator_worker_builder.add_edge(START,"orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator",assign_workers,[llm_call]
)
orchestrator_worker_builder.add_edge("llm_call","synthesizer")
orchestrator_worker_builder.add_edge("synthesizer",END)

# Compile the workflow
orchestrator_worker=orchestrator_worker_builder.compile()

# show the workflow
display(Image(orchestrator_worker.get_graph().draw_mermaid_png()))

ValueError: At 'orchestrator' node, 'assign_workers' branch found unknown target '<function llm_call at 0x00000272D255BBA0>'